# Instrument view

## Overview

A simple version of the Mantid [instrument view](https://www.mantidproject.org/MantidPlot:_Instrument_View) is available in `scipp`.
It currently does not support detector 'picking' and manual drawing of masks,
nor does it render the actual shape of the detectors (currently it represents them as 2d squares),
but basic functionalities such as spatial slicing,
as well as navigation through the time-of-flight dimension via a slider, are provided.

<div class="alert alert-info">

**Note**

The files used in this notebook were taken from the Mantid [training course data](http://sourceforge.net/projects/mantid/files/Sample%20Data/TrainingCourseData.zip/download).

</div>

<div class="alert alert-warning">

**Warning**
    
While you can interact with the 3D view of the instrument,
the buttons and sliders will have no effect in the documentation pages,
as there is no kernel to perform the operations.
These will only work inside a Jupyter notebook.

</div>

The instrument view is part of the `scippneutron` module, and it is launched as follows:

In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn

In [ ]:
sample = scn.load(scn.data.get_path('PG3_4844_event.nxs'))
sample

In [ ]:
scn.instrument_view(sample)

You can adjust opacities and create a cut surface to slice your data in 3D using the control buttons below the scene.

It is possible to customize the figure using the usual arguments, as well as adjusting the pixel size, e.g.

In [ ]:
scn.instrument_view(
    sample, cmap="magma", vmax=2000.0 * sc.units.counts, norm="log", pixel_size=0.03
)

This works for any file that can be loaded by Mantid that contains a valid instrument description or geometry:

In [ ]:
gem_da = scn.load(scn.data.get_path('GEM40979.raw'))
scn.instrument_view(gem_da, pixel_size=0.05)

## Displaying additional components

Additional beamline components can be displayed in the instrument view by providing a `dict` of component settings.
The key for the component is used as the text label.
Component settings are given as a settings dictionary defining, e.g., `color`, `size`, and `type`.

First let us see what we could additionally plot:

In [ ]:
gem_da

We can see `source_position` and `sample_position` stored as 3d vectors.
In this case we want to call our `source_position` *source*.
We can pick from a limited number of `type`s including `cylinder`, `disk` and `box`:

In [ ]:
sample_settings = {
    'center': gem_da.meta['sample_position'],
    'color': '#000000',
    'wireframe': True,
    'size': sc.vector(value=[0.5, 0.5, 0.5], unit=sc.units.m),
    'type': 'box',
}
source_settings = {
    'center': gem_da.meta['source_position'],
    'color': '#FFC133',
    'size': sc.vector(value=[1000, 2000, 1000], unit=sc.units.mm),
    'type': 'cylinder',
}
scn.instrument_view(
    gem_da,
    pixel_size=0.05,
    components={'sample': sample_settings, 'source': source_settings},
)

## Defining detector positions

If data comes without pre-defined detector-positions, e.g., when they are not contained in the original file, they can be defined by simply adding a `position` coordinate a a data array:

In [ ]:
nx, ny = (10, 10)
points_x = np.linspace(0, 1, nx)
points_y = np.linspace(0, 1, ny)
xv, yv = np.meshgrid(points_x, points_y)

_x = sc.array(dims=['spectrum'], values=xv.ravel())
_y = sc.array(dims=['spectrum'], values=yv.ravel())
_z = sc.array(dims=['spectrum'], values=np.zeros(100))

data = sc.array(dims=['spectrum', 'tof'], values=np.arange(1000).reshape(100, 10))
da = sc.DataArray(data)
da.coords['position'] = sc.geometry.position(_x, _y, _z)
da.coords['tof'] = sc.arange('tof', 0, 100, 10)
da.coords['spectrum'] = sc.arange('spectrum', 100)

scn.instrument_view(da, pixel_size=0.1)